
# Основы PyTorch: Работа с тензорами, устройствами и операцией свёртки

В этом ноутбуке мы познакомимся с базовой работой в PyTorch, включая:

- создание и манипуляции с тензорами
- выбор устройства выполнения (CPU, CUDA, MPS)
- реализация одномерной и двумерной свёртки

---

## Устройства: CPU, CUDA, MPS


In [2]:
import torch

# Определим доступное устройство
device = (
    torch.device("cuda") if torch.cuda.is_available() 
    else torch.device("mps") if torch.backends.mps.is_available() 
    else torch.device("cpu")
)
print(f"Используемое устройство: {device}")


Используемое устройство: mps



## Работа с тензорами

Тензоры — это базовые структуры данных в PyTorch, аналогичные массивам NumPy, но с поддержкой GPU.


In [8]:
def mult_two_matrices(n, device):
    a = torch.randn(n, n, device=device)
    b = torch.randn(n, n, device=device)
    c = a @ b
    return c

for d in ['cpu', device]:
    print(f"Устройство: {d}")
    %timeit mult_two_matrices(1000, d)

Устройство: cpu
14.3 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Устройство: mps
472 µs ± 956 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)



## Операция свёртки

Свёртка — это операция, которая часто используется в обработке сигналов и компьютерном зрении.

### Одномерная свёртка:

Пусть $x = [x_0, x_1, \ldots, x_{n-1}]$ — входной сигнал, $w = [w_0, w_1, \ldots, w_{k-1}]$ — фильтр.

Тогда одномерная свёртка $\hat{x}$ определяется как:

$$
\hat{x}[i] = \sum_{j=0}^{k-1} x[i + j] \cdot w[j]
$$

Например, если $x = [1, 2, 3, 4, 5]$ и $w = [-1, 2, 1]$, то:

$$
\hat{x}_1 = -1 \cdot 1 + 2 \cdot 2 + 1 \cdot 3 = 6 \\
\hat{x}_2 = -1 \cdot 2 + 2 \cdot 3 + 1 \cdot 4 = 8 \\
\hat{x}_3 = -1 \cdot 3 + 2 \cdot 4 + 1 \cdot 5 = 10
$$



In [22]:
x = torch.tensor([1, 2, 3, 4, 5]).unsqueeze(0).unsqueeze(0)
w = torch.tensor([-1, 2, 1]).unsqueeze(0).unsqueeze(0)
y = torch.nn.functional.conv1d(x, w)
print(y)


tensor([[[ 6,  8, 10]]])


`torch.nn.functional.conv1d` используется для выполнения одномерной свёртки.

`x` и `w` должны иметь форму `(1, 1, n)`, где `n` — длина входного сигнала.

`conv1d` принимает на вход два тензора, а также имеет важные параметры:

- `stride` — шаг свёртки
- `padding` — количество нулевых значений, добавляемых к входному сигналу

Например, если `stride = 2` и `padding = 1`, то:

$$
\hat{x}[i] = \sum_{j=0}^{k-1} x[i + j] \cdot w[j]
$$

В нашем случае:

Так как `padding = 1`, то мы добавляем 1 нуль слева и справа от входного сигнала:

$$
x \mapsto [0, 1, 2, 3, 4, 5, 0]
$$

Теперь мы можем выполнить свёртку, используя `stride = 2`, то есть сдвигая фильтр на 2 позиции:

$$
\hat{x}_1 = -1 \cdot 0 + 2 \cdot 1 + 1 \cdot 2 = 4 \\
\hat{x}_2 = -1 \cdot 2 + 2 \cdot 3 + 1 \cdot 4 = 8 \\
\hat{x}_3 = -1 \cdot 4 + 2 \cdot 5 + 1 \cdot 0 = 6
$$







In [23]:
torch.nn.functional.conv1d(x, w, stride=2, padding=1)

tensor([[[4, 8, 6]]])

### Двумерная свёртка

Двумерная свёртка — это операция, которая часто используется в компьютерном зрении и обработке изображений.

Пусть $x = \begin{bmatrix}
x_{00} & \ldots & x_{0n} \\
\vdots & \ddots & \vdots \\
x_{m0} & \ldots & x_{mn}
\end{bmatrix}$ — входное изображение, $w = \begin{bmatrix}
w_{00} & \ldots & w_{0a} \\
\vdots & \ddots & \vdots \\
w_{b0} & \ldots & w_{ba}
\end{bmatrix}$ — фильтр.

Тогда двумерная свёртка $\hat{x}$ определяется как:

$$
\hat{x}_{ij} = \sum_{a=0}^{a} \sum_{b=0}^{b} x_{i+a,j+b} \cdot w_{ab}
$$

Например, если $x = \begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{bmatrix}$ и $w = \begin{bmatrix}
-1 & 2 \\
1 & 0
\end{bmatrix}$, то:

$$
\hat{x}_{00} = -1 \cdot 1 + 2 \cdot 2 + 1 \cdot 4 + 0 \cdot 5 = 7 \\
\hat{x}_{01} = -1 \cdot 2 + 2 \cdot 3 + 1 \cdot 5 + 0 \cdot 6 = 9 \\
\hat{x}_{10} = -1 \cdot 4 + 2 \cdot 5 + 1 \cdot 7 + 0 \cdot 8 = 13 \\
\hat{x}_{11} = -1 \cdot 5 + 2 \cdot 6 + 1 \cdot 8 + 0 \cdot 9 = 15
$$


In [29]:
x = torch.arange(1, 10).reshape(1, 1, 3, 3)  # Shape: (batch_size, channels, height, width)
w = torch.tensor([[-1, 2], [1, 0]]).reshape(1, 1, 2, 2)  # Shape: (out_channels, in_channels, kernel_height, kernel_width)
y = torch.nn.functional.conv2d(x, w, stride=1, padding=0)
print(y)

tensor([[[[ 7,  9],
          [13, 15]]]])
